### Obtenición de ángulos en flechas sobre el pavimento

Utilizar una red YOLO-OBB no nos sirve porque no obtendríamos la orientación, sino el rectángulo rotado que mejor encaja con el objeto.

La propuesta para obtener la orientación de las flechas consiste en diseñar una CNN ad-hoc que entrenaremos con las imágenes de las flechas extraidas por YOLO y la rotación proporcionada en los labels.

Para ello se eliminarán todas las imágenes que no tienen flechas, y de las que sí tienen se recortará cada flecha (puede haber varias y se reescalará a unas dimensiones fijas (por determinar), añadiendo el ángulo reportado en los labels.

Con ese dataset se entrenará una red tipo VGG para obtener dada una imagen de una flecha el ángulo que forma.

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#!pip install opencv-python

In [5]:
import os
import pandas as pd
import numpy as np
import cv2

import matplotlib.pyplot as plt

def load_samples (directory, size=None):
    data = pd.read_csv(directory, sep='\t', header=None) 

    #data = data[:10]
    print(data)
    
    X = []
    Y = []
    for i in range(len(data)):
        print('.', end='')
        # carga la imagen
        imgname = data.iloc[i,0]
        img = cv2.imread(imgname)
        if size != None:
            img = cv2.resize(img, size)
        X.append(img)
        Y.append(data.iloc[i,1])

    return (np.array(X), np.array(Y))

In [6]:
X, Y = load_samples("dataset_cleaned.csv",(64,64)) # for model that require higher than 64x64 resolution

                                                   0         1
0        dataset/lyrs=y&x=498928&y=388383&z=20-0.png  0.747256
1        dataset/lyrs=y&x=498933&y=388386&z=20-1.png  0.085187
2        dataset/lyrs=y&x=498934&y=388391&z=20-2.png  0.634782
3        dataset/lyrs=y&x=498934&y=388391&z=20-3.png  0.140102
4        dataset/lyrs=y&x=498940&y=388395&z=20-4.png  0.887765
...                                              ...       ...
6696  dataset/lyrs=y&x=530663&y=391686&z=20-7116.png  0.514914
6697  dataset/lyrs=y&x=530669&y=391622&z=20-7117.png  0.385349
6698  dataset/lyrs=y&x=530672&y=391647&z=20-7118.png  0.652626
6699  dataset/lyrs=y&x=530676&y=391636&z=20-7119.png  0.598877
6700  dataset/lyrs=y&x=530680&y=391631&z=20-7120.png  0.615035

[6701 rows x 2 columns]
...........................................................................................................................................................................................................................

In [7]:
print(X.shape)
print(Y.shape)

# get train and test sets
import sklearn.model_selection

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.10, random_state=42) # this random_state is similar to the bootstrap results

(6701, 64, 64, 3)
(6701,)


In [8]:
# imprime algunos ejemplos

N = 16

fig, axs = plt.subplots(N//4,4,figsize=(10, 10*N//16))
axs = axs.flatten()
plt.axis('off')

im0 = 567
for i in range(N):
    axs[i].axis('off')
    axs[i].imshow(X[im0+i])
    #axs[i].title.set_text(f"$angle$ = {(Y[im0+i]):.3f}")
    axs[i].title.set_text(f"$angle$ = {(360*Y[im0+i]):.1f}°")

#plt.savefig("test.png", bbox_inches='tight')